In [1]:
#this file performs the unzipping of the files, it creates a single data frame for every asset and saves it in the data folder
import os
import pandas as pd
import gzip
import tarfile
import xlrd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from functions import *
import re
import dask
import vaex
import glob
from numpy import linalg as LA

In [ ]:
# done 
# stock_tickers contains the name of all the files/stocks
stock_tickers_bbo = get_file_names('data/raw/sp100_2004-8/bbo')
stock_tickers_bbo.remove('.DS_Store') 
for file_name_bbo in stock_tickers_bbo:
    file_path_bbo = f"data/raw/sp100_2004-8/bbo/{file_name_bbo}/{file_name_bbo}_bbo.tar"
    output_path_bbo = f"data/raw/sp100_2004-8/bbo/{file_name_bbo}/"
    extract_tar(file_path_bbo, output_path_bbo)

In [ ]:
# done 
# I need to do that 
stock_tickers_trade = get_file_names('data/raw/sp100_2004-8/trade')
# stock_tickers_trade.remove('.DS_Store') 
for file_name_trade in stock_tickers_trade:
    file_path_trade = f"data/raw/sp100_2004-8/trade/{file_name_trade}/{file_name_trade}_trade.tar"
    output_path_trade = f"data/raw/sp100_2004-8/trade/{file_name_trade}/"
    extract_tar(file_path_trade, output_path_trade)

In [ ]:
tickers = get_file_names("data/raw/sp100_2004-8/trade/")
# tickers.remove('.DS_Store') 
tickers.remove('MSFT.O')
tickers.remove('ORCL.N') 

In [ ]:
# done
# don't do that with dask
for ticker in tickers:
    data_to_parquet(ticker)

In [2]:
data = pd.read_parquet("data/clean/sp100_2004-8/AA.N.parquet")

In [3]:
data

,trade_price,trade_volume,bid-price,bid-volume,ask-price,ask-volume
xltime,,,,,,
2004-01-02 09:30:51.038000-05:00,38.00,600.0,38.00,1.0,38.05,1.0
2004-01-02 09:31:03.359999872-05:00,38.00,5000.0,38.00,1.0,38.05,1.0
2004-01-02 09:31:36.730000256-05:00,38.05,6000.0,38.06,2.0,38.08,1.0
2004-01-02 09:31:41.369000192-05:00,38.06,1500.0,38.06,4.0,38.08,1.0
2004-01-02 09:31:58.910000256-05:00,38.06,500.0,38.04,1.0,38.07,1.0
...,...,...,...,...,...,...
2008-12-31 15:59:58.225000192-05:00,11.26,100.0,11.25,72.0,11.26,66.0
2008-12-31 15:59:58.387000064-05:00,11.25,100.0,11.25,86.0,11.26,87.0
2008-12-31 15:59:58.431000064-05:00,11.26,200.0,11.25,86.0,11.26,87.0


In [4]:
data.isnull().sum()

trade_price     0
trade_volume    0
bid-price       0
bid-volume      0
ask-price       0
ask-volume      0
dtype: int64

In [5]:
data1 = data.copy()
data1.drop(columns=['trade_volume', 'bid-volume', 'ask-volume'], inplace=True)

In [6]:
data1

,trade_price,bid-price,ask-price
xltime,,,
2004-01-02 09:30:51.038000-05:00,38.00,38.00,38.05
2004-01-02 09:31:03.359999872-05:00,38.00,38.00,38.05
2004-01-02 09:31:36.730000256-05:00,38.05,38.06,38.08
2004-01-02 09:31:41.369000192-05:00,38.06,38.06,38.08
2004-01-02 09:31:58.910000256-05:00,38.06,38.04,38.07
...,...,...,...
2008-12-31 15:59:58.225000192-05:00,11.26,11.25,11.26
2008-12-31 15:59:58.387000064-05:00,11.25,11.25,11.26
2008-12-31 15:59:58.431000064-05:00,11.26,11.25,11.26


In [7]:
data1.isnull().sum()

trade_price    0
bid-price      0
ask-price      0
dtype: int64

In [8]:
data1 = data1.resample('1T').mean()

In [9]:
data1

,trade_price,bid-price,ask-price
xltime,,,
2004-01-02 09:30:00-05:00,38.000000,38.000000,38.050000
2004-01-02 09:31:00-05:00,38.042500,38.040000,38.070000
2004-01-02 09:32:00-05:00,38.038750,38.006250,38.062500
2004-01-02 09:33:00-05:00,38.118333,38.095000,38.155000
2004-01-02 09:34:00-05:00,38.126154,38.090000,38.160769
...,...,...,...
2008-12-31 15:55:00-05:00,11.370385,11.362596,11.373173
2008-12-31 15:56:00-05:00,11.358115,11.352131,11.364590
2008-12-31 15:57:00-05:00,11.333291,11.325696,11.340886


In [10]:
data1 = data1.dropna()
data1

,trade_price,bid-price,ask-price
xltime,,,
2004-01-02 09:30:00-05:00,38.000000,38.000000,38.050000
2004-01-02 09:31:00-05:00,38.042500,38.040000,38.070000
2004-01-02 09:32:00-05:00,38.038750,38.006250,38.062500
2004-01-02 09:33:00-05:00,38.118333,38.095000,38.155000
2004-01-02 09:34:00-05:00,38.126154,38.090000,38.160769
...,...,...,...
2008-12-31 15:55:00-05:00,11.370385,11.362596,11.373173
2008-12-31 15:56:00-05:00,11.358115,11.352131,11.364590
2008-12-31 15:57:00-05:00,11.333291,11.325696,11.340886


In [9]:
data1.isnull().sum()

trade_price    0
bid-price      0
ask-price      0
dtype: int64

In [ ]:
(data1.shift(1)/data1-1).isnull().sum()

In [ ]:
returns = (data1.shift(1) / data1 - 1).dropna()
returns

In [ ]:
type(returns.std())

## I want to create a dataframe in which the columns are the trading prices returns 

In [1]:
#this file performs the unzipping of the files, it creates a single data frame for every asset and saves it in the data folder
import os
import pandas as pd
import gzip
import tarfile
import xlrd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from functions import *
import re
import dask
import vaex
import glob
from numpy import linalg as LA

In [2]:
trading_returns = pd.DataFrame()
tickers = get_file_names("data/clean/sp100_2004-8/")
tickers.remove('resampled')

In [3]:
for ticker in tickers: 
    df = pd.read_parquet(f"data/clean/sp100_2004-8/{ticker}")
    df = df.resample('1T').mean().dropna()
    df_prices = df.drop(columns=['trade_volume', 'bid-volume', 'ask-volume'])
    df_returns = (df_prices / df_prices.shift(1) - 1).dropna() # do we want log returns for some reason? 
    # df_returns.to_parquet(f"data/clean/sp100_2004-8/resampled/")
    trading_returns = pd.concat([trading_returns, df_returns['trade_price'].to_frame(name= ticker[:-8])], axis=1, join='outer')
trading_returns.dropna(inplace=True)

AA.N.parquet is done
ABT.N.parquet is done
AEP.N.parquet is done
ALL.N.parquet is done
APA.N.parquet is done
AVP.N.parquet is done
AXP.N.parquet is done
BA.N.parquet is done
BAC.N.parquet is done
BAX.N.parquet is done
BHI.N.parquet is done
BK.N.parquet is done
BMY.N.parquet is done
C.N.parquet is done
CAT.N.parquet is done
CL.N.parquet is done
COF.N.parquet is done
COP.N.parquet is done
CVS.N.parquet is done
CVX.N.parquet is done
DD.N.parquet is done
DIS.N.parquet is done
DOW.N.parquet is done
DVN.N.parquet is done
EMC.N.parquet is done
EMR.N.parquet is done
ETR.N.parquet is done
EXC.N.parquet is done
F.N.parquet is done
FCX.N.parquet is done
FDX.N.parquet is done
GD.N.parquet is done
GE.N.parquet is done
GS.N.parquet is done
HAL.N.parquet is done
HD.N.parquet is done
HNZ.N.parquet is done
HON.N.parquet is done
HPQ.N.parquet is done
IBM.N.parquet is done
JNJ.N.parquet is done
JPM.N.parquet is done
KFT.N.parquet is done
KO.N.parquet is done
LMT.N.parquet is done
LOW.N.parquet is done
MA

In [4]:
trading_returns

,AA.N,ABT.N,AEP.N,ALL.N,APA.N,AVP.N,AXP.N,BA.N,BAC.N,BAX.N,...,UTX.N,V.N,VZ.N,WAG.N,WFC.N,WMB.N,WMT.N,WY.N,XOM.N,XRX.N
xltime,,,,,,,,,,,,,,,,,,,,,
2008-03-31 09:34:00-04:00,-0.003108,-0.003439,-0.001023,-0.000344,0.002849,0.000000,-0.003360,-0.004431,-0.002020,-0.002459,...,-0.001455,-0.000682,-0.000928,-0.000616,0.000459,0.001662,-0.002556,0.000861,0.000027,-0.003516
2008-03-31 09:35:00-04:00,0.001304,-0.003720,0.000244,-0.001374,-0.002221,-0.001999,0.000612,0.001238,-0.001373,-0.001908,...,0.000590,-0.000817,-0.000287,-0.000076,-0.000876,-0.000156,-0.000761,-0.001994,-0.000756,-0.003079
2008-03-31 09:36:00-04:00,-0.000386,-0.003254,-0.000465,0.000559,0.001963,0.000801,0.002286,0.000849,-0.000762,-0.003070,...,0.001305,0.000277,0.000748,0.000022,-0.000619,0.000143,0.000170,0.003794,0.000113,-0.002068
2008-03-31 09:37:00-04:00,0.000928,-0.001382,-0.002479,-0.000196,0.002669,-0.000307,0.001281,-0.000066,-0.000655,-0.000472,...,-0.000554,0.000229,-0.000358,0.000096,-0.000111,-0.000394,-0.000186,0.001117,-0.000187,-0.000537
2008-03-31 09:38:00-04:00,-0.000336,0.002408,0.000141,-0.000078,0.004070,-0.000613,0.000639,-0.000436,0.000456,0.000417,...,0.000520,0.000091,0.000468,-0.000015,0.000382,0.000657,0.000105,-0.000789,0.001147,0.002441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 15:55:00-05:00,0.003036,0.001525,0.000790,0.002557,0.001589,0.002436,0.003339,0.002210,0.008263,0.001360,...,0.001918,0.001384,0.000879,0.002734,0.004390,0.001111,0.001130,0.001792,0.000418,0.002216
2008-12-31 15:56:00-05:00,-0.001079,0.002034,0.001647,0.000885,0.000495,0.002354,0.001546,0.000988,0.002100,0.002341,...,0.000251,0.002322,0.001894,0.002173,0.002008,0.002196,0.001909,0.000464,0.002314,0.002090
2008-12-31 15:57:00-05:00,-0.002186,0.001082,-0.000120,-0.000378,0.000838,0.000409,0.001382,0.001218,0.002511,-0.000321,...,0.000181,0.000439,0.001190,0.000029,0.001810,0.000775,0.000314,-0.000384,0.000553,-0.000474


In [6]:
def process_parquet_files(ticker, trading_returns):
    if ~os.path.exists(f"data/clean/sp100_2004-8/resampled/{ticker}"):
        df = pd.read_parquet(f"data/clean/sp100_2004-8/{ticker}")
        df = df.resample('1T').mean().dropna()
        df_prices = df.drop(columns=['trade_volume', 'bid-volume', 'ask-volume'])
        df_returns = (df_prices / df_prices.shift(1) - 1).dropna() # do we want log returns for some reason? 
        # df_returns.to_parquet(f"data/clean/sp100_2004-8/resampled/")
        trading_returns = pd.concat([trading_returns, df_returns['trade_price'].to_frame(name= ticker[:-8])], axis=1, join='outer')
    trading_returns.dropna(inplace=True)
    return trading_returns

In [7]:
def covariance_matrix(trading_returns_df): 
    # number of timesteps
    T = trading_returns_df.shape[0] 
    initial_corr = trading_returns_df.corr()
    # number of assets
    N = initial_corr.shape[0] 
    if N>T: 
        print("N is bigger than T and risk estimation error may diverge")
    q = N/T
    eigenvalues_e, eigenvectors_e = LA.eig(initial_corr)
    lambda_plus = (1+np.sqrt(q))**2
    # number of eigenvalues outside of the random bulk
    outside_bulk_eigenvalues = np.sum(eigenvalues_e>lambda_plus)
    corr_clipped = eigenvalue_clipping(eigenvalues_e,eigenvectors_e,lambda_plus)
    # from correlation matrix to covariance matrix
    trading_std = np.array(trading_returns_df.std())
    cov_matrix = np.outer(trading_std, trading_std) * np.array(corr_clipped)
    return corr_clipped, cov_matrix, outside_bulk_eigenvalues

In [8]:
covariance_matrix(trading_returns)

(matrix([[ 1.        , -0.47663216, -0.17653461, ...,  0.8966004 ,
          -1.43983117,  0.63163884],
         [-0.47663216,  1.        ,  0.31049811, ..., -0.96816983,
           1.52674206, -0.63043201],
         [-0.17653461,  0.31049811,  1.        , ..., -0.47551445,
           0.73065003, -0.28977397],
         ...,
         [ 0.8966004 , -0.96816983, -0.47551445, ...,  1.        ,
          -2.69083595,  1.13528631],
         [-1.43983117,  1.52674206,  0.73065003, ..., -2.69083595,
           1.        , -1.81107537],
         [ 0.63163884, -0.63043201, -0.28977397, ...,  1.13528631,
          -1.81107537,  1.        ]]),
 array([[ 4.27625715e-06, -1.00168217e-06, -4.48099527e-07, ...,
          3.19513230e-06, -3.99123768e-06,  1.85372376e-06],
        [-1.00168217e-06,  1.03283129e-06,  3.87334671e-07, ...,
         -1.69560330e-06,  2.07990969e-06, -9.09279145e-07],
        [-4.48099527e-07,  3.87334671e-07,  1.50669531e-06, ...,
         -1.00585285e-06,  1.20222633e-06, 

In [ ]:
tickers = get_file_names("data/clean/sp100_2004-8/")

In [ ]:
for ticker in tickers:
    process_parquet_files(ticker)